# Tutorial 5: General Relativity with MechanicsDSL

This tutorial explores particle motion and light propagation near black holes
using MechanicsDSL's general relativity module.

In [ ]:
from mechanics_dsl import PhysicsCompiler
from mechanics_dsl.domains.general_relativity import (
    SchwarzschildMetric, KerrMetric, Geodesic, GravitationalLensing
)
import matplotlib.pyplot as plt
import numpy as np

## 1. The Schwarzschild Black Hole

The simplest black hole: non-rotating, uncharged, described by the Schwarzschild metric.

### Key Parameters

- **Schwarzschild radius**: $r_s = \frac{2GM}{c^2}$ (event horizon)
- **Photon sphere**: $r = 1.5 r_s$ (light can orbit)
- **ISCO**: $r = 3 r_s$ (innermost stable circular orbit)

In [ ]:
# Create a Schwarzschild black hole
bh = SchwarzschildMetric(M=1.0)  # 1 solar mass

print("Schwarzschild Black Hole (1 Solar Mass)")
print("=" * 45)
print(f"Schwarzschild radius: r_s = {bh.r_s:.3f} km")
print(f"Photon sphere: r = {1.5 * bh.r_s:.3f} km")
print(f"ISCO (prograde): r = {bh.isco():.3f} km")

## 2. Effective Potential for Orbital Motion

The effective potential determines allowed orbits:

$$V_{\text{eff}}(r) = -\frac{GM}{r} + \frac{L^2}{2r^2} - \frac{GML^2}{c^2 r^3}$$

The last term is the relativistic correction responsible for:
- Perihelion precession (Mercury's famous 43"/century)
- The existence of an unstable photon orbit
- Plunging orbits near the event horizon

In [ ]:
# Plot effective potential for different angular momenta
r = np.linspace(2.1, 20, 500)  # In units of r_s

fig, ax = plt.subplots(figsize=(10, 6))

L_values = [3.0, 3.5, 4.0, 4.5, 5.0]  # Angular momentum in units of M*c

for L in L_values:
    V_eff = bh.effective_potential(r, L)
    ax.plot(r, V_eff, label=f'L = {L} Mc')

ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax.axvline(x=2, color='r', linestyle='--', alpha=0.5, label='Event horizon')
ax.axvline(x=3, color='orange', linestyle='--', alpha=0.5, label='Photon sphere')
ax.axvline(x=6, color='g', linestyle='--', alpha=0.5, label='ISCO')

ax.set_xlabel('r / r_s')
ax.set_ylabel('V_eff / (mc²)')
ax.set_title('Effective Potential in Schwarzschild Spacetime')
ax.set_xlim(2, 20)
ax.set_ylim(-0.1, 0.05)
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 3. Geodesics: Particle Orbits

Massive particles follow timelike geodesics. Let's trace some orbits.

In [ ]:
# Trace a stable elliptical orbit
geodesic = Geodesic(bh, geodesic_type='timelike')

# Initial conditions: r=10 r_s, starting at perihelion
r0 = 10.0       # Starting radius (in r_s)
phi0 = 0.0      # Starting angle
E = 0.95        # Specific energy (E/mc²)
L = 4.0         # Specific angular momentum (L/Mc)

# Integrate the geodesic equations
tau, r, phi = geodesic.integrate(
    r0=r0, phi0=phi0, E=E, L=L,
    tau_max=500, n_points=5000
)

# Convert to Cartesian for plotting
x = r * np.cos(phi)
y = r * np.sin(phi)

fig, ax = plt.subplots(figsize=(10, 10))

# Draw black hole
circle_horizon = plt.Circle((0, 0), 2, color='black', label='Event horizon')
circle_photon = plt.Circle((0, 0), 3, color='orange', fill=False, linestyle='--', label='Photon sphere')
circle_isco = plt.Circle((0, 0), 6, color='green', fill=False, linestyle='--', label='ISCO')
ax.add_patch(circle_horizon)
ax.add_patch(circle_photon)
ax.add_patch(circle_isco)

# Plot orbit
ax.plot(x, y, 'b-', lw=0.5, alpha=0.7)
ax.plot(x[0], y[0], 'go', markersize=10, label='Start')

ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)
ax.set_aspect('equal')
ax.set_xlabel('x / r_s')
ax.set_ylabel('y / r_s')
ax.set_title('Precessing Orbit Around a Schwarzschild Black Hole')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.show()

# Calculate precession per orbit
precession = geodesic.perihelion_precession(E, L)
print(f"Perihelion precession: {np.degrees(precession):.4f}° per orbit")

## 4. Light Bending and Gravitational Lensing

Light rays follow null geodesics. Near massive objects, they bend.

### Deflection Angle

For a light ray passing at closest approach $r_0$:

$$\Delta\phi = \frac{4GM}{c^2 r_0} = \frac{2r_s}{r_0}$$

Einstein predicted this in 1915. It was confirmed in 1919.

In [ ]:
# Calculate light deflection
lensing = GravitationalLensing(bh)

# Light ray passing at various closest approach distances
r_min_values = np.linspace(3, 100, 100)  # In units of r_s
deflections = [lensing.deflection_angle(r_min) for r_min in r_min_values]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(r_min_values, np.degrees(deflections), 'b-', lw=2)
ax.set_xlabel('Closest approach distance (r_s)')
ax.set_ylabel('Deflection angle (degrees)')
ax.set_title('Gravitational Light Bending')
ax.grid(True, alpha=0.3)

# Mark some reference points
r_sun = 100  # Grazing sun (approximately)
ax.axhline(y=np.degrees(lensing.deflection_angle(r_sun)), color='orange', 
           linestyle='--', alpha=0.5, label=f'Sun (r≈100 r_s): {np.degrees(lensing.deflection_angle(r_sun)):.4f}°')
ax.legend()
plt.show()

In [ ]:
# Trace multiple light rays to visualize lensing
null_geodesic = Geodesic(bh, geodesic_type='null')

fig, ax = plt.subplots(figsize=(12, 8))

# Draw black hole
circle_horizon = plt.Circle((0, 0), 2, color='black')
circle_photon = plt.Circle((0, 0), 3, color='orange', fill=False, linestyle='--')
ax.add_patch(circle_horizon)
ax.add_patch(circle_photon)

# Trace light rays coming from the left
impact_params = [-8, -6, -4, -3.5, 3.5, 4, 6, 8]

for b in impact_params:
    # Initial conditions for light ray with impact parameter b
    r0 = 50  # Start far away
    phi0 = np.pi  # Coming from the left
    
    try:
        _, r, phi = null_geodesic.integrate_from_infinity(b=b, n_points=1000)
        x = r * np.cos(phi)
        y = r * np.sin(phi)
        ax.plot(x, y, 'b-', lw=1, alpha=0.7)
    except:
        pass  # Ray captured by black hole

ax.set_xlim(-30, 30)
ax.set_ylim(-15, 15)
ax.set_aspect('equal')
ax.set_xlabel('x / r_s')
ax.set_ylabel('y / r_s')
ax.set_title('Light Ray Trajectories Near a Black Hole')
ax.grid(True, alpha=0.3)
plt.show()

## 5. Einstein Ring

When a source, lens, and observer are perfectly aligned, the source appears as a ring.

In [ ]:
# Calculate Einstein ring radius
D_L = 1e9   # Lens distance (meters) — example
D_S = 2e9   # Source distance (meters)
D_LS = D_S - D_L  # Distance from lens to source

theta_E = lensing.einstein_radius(D_L, D_S, D_LS)

print(f"Einstein Ring Calculation")
print(f"=" * 40)
print(f"Lens distance: D_L = {D_L:.2e} m")
print(f"Source distance: D_S = {D_S:.2e} m")
print(f"Einstein radius: θ_E = {np.degrees(theta_E)*3600:.4f} arcseconds")

## 6. Using the DSL for GR Simulations

You can also define relativistic systems using the MechanicsDSL notation.

In [ ]:
dsl_code = r"""
\system{schwarzschild_orbit}

\parameter{M}{1.0}{solar_mass}
\parameter{c}{299792458}{m/s}
\parameter{G}{6.674e-11}{N*m^2/kg^2}

\defvar{r}{radial coordinate}{m}
\defvar{phi}{azimuthal angle}{rad}

\metric{schwarzschild}
\geodesic{timelike}

\initial{r=10*r_s, phi=0}
\initial{E=0.95, L=4.0}
"""

compiler = PhysicsCompiler()
result = compiler.compile_dsl(dsl_code)

if result['success']:
    print("GR system compiled successfully!")
    print(f"Metric: {result.get('metric_type', 'Schwarzschild')}")

## Summary

In this tutorial, we explored:

1. **Schwarzschild Black Holes** — Event horizon, photon sphere, ISCO
2. **Effective Potential** — Orbital dynamics in curved spacetime
3. **Geodesics** — Precessing orbits around black holes
4. **Light Bending** — Gravitational lensing and deflection angles
5. **Einstein Rings** — Perfect alignment lensing

## Next Steps

- **Tutorial 6**: Statistical Mechanics (partition functions, ensembles)